# Proyecto: Manejo de base de datos con SQL
### Autor: Rodrigo Iván González Valenzuela
<hr>
Para la materia de Introducción a la Ciencia de Datos y sus Metodologías impartida por el Dr. Juan Pablo Soto Barrera.

Maestría en Ciencia de Datos

Universidad de Sonora

*División de Ciencias Exactas y Naturales*
<hr>

Esta libreta contine el cógido y procedimiento empleado para la obtención del mapa de incidencia de homicidios por municipio de Sonora presentado en el reporte del proyecto.

Los datos son obtenidos de una base de datos, en un servidor local de MySQL, descrita en el documento mencionado.

`pymysql` es una librería cliente de MySQL para realizar conexiones a una base de datos, donde un cursor se utiliza para gestionar el contexto de la operación de consulta.

In [108]:
import pymysql.cursors
import pandas as pd

Para acceder a la base de datos, hay que definir un *Connection Object* con el método `connect()`: 

In [111]:
connection=pymysql.connect(host='localhost',
                             user='root',
                             password=__YOUR_PASSWORD__,
                             database='crimen_mexico',
                             cursorclass=pymysql.cursors.DictCursor)

Si la conexión fue exitosa, procedemos a almacenar la consulta (escrita en SQL) en un dataframe de `pandas`:

In [112]:
query='SELECT * FROM sonora_delitos'
sql_df=pd.read_sql(query, connection, index_col='ID_DELITO')

<hr>

Para no alterar el dataframe con la consulta, realizamos una copia y filtramos los registros con cualquier tipo de homicidio, agrupamos la suma por municipio y omitimos los datos que no fueron registrados en alguna locación.

In [113]:
df=sql_df.copy(deep=True)
df=df[df['TIPO']=='Homicidio']
df=df[['ID_LOCACION','MUNICIPIO','TOTAL']]
df=df.groupby(['ID_LOCACION','MUNICIPIO']).sum()
df.reset_index(inplace=True)
df=df[df['ID_LOCACION']!=26998]
df=df[df['ID_LOCACION']!=26999]
df['ID_LOCACION']=df['ID_LOCACION'].astype(str)

Para graficar la geometría de los territorios, importamos los datos georeferenciados usando el paquete `geopandas` y los añadimos al dataframe antes creado. 

In [114]:
import geopandas as gpd

gdf=gpd.read_parquet('data/shapes_gdframe.parquet')
son=df.merge(gdf[['ID','geometry']], how='outer', left_on='ID_LOCACION', right_on='ID')

Finalmante, `folium` nos permitirá visualizar nuestros datos en un mapa leaflet interactivo.

In [115]:
import folium as fl

map=fl.Map(location=[29.68770,-110.80995],
    tiles='Stamen Toner',
    zoom_start=7,
    zoomControl=False, doubleClickZoom=False,
    dragging=False, scrollWheelZoom=False,
    attributionControl=False)

choro=fl.Choropleth(
    geo_data=gdf,
    data=df,
    columns=['ID_LOCACION', 'TOTAL'],
    key_on='feature.properties.ID',
    fill_color='YlOrRd',
    fill_opacity=0.8,
    line_opacity=1.0,
    legend_name='Homicidios en Sonora',
    highlight=True,
    line_color='black').add_to(map)

map